In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import argparse
from bertax.utils import (seq2tokens, get_token_dict, process_bert_tokens_batch, load_bert,
                          parse_fasta, annotate_predictions, best_predictions, seq_frames,
                          CLASS_LABELS)
import tensorflow as tf
from logging import  info, getLogger, INFO, WARNING
import numpy as np
from random import sample
import json
import pkg_resources


/tmp/ipykernel_1148547/4085345833.py:14: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
model_file = pkg_resources.resource_filename('bertax', 'resources/big_trainingset_all_fix_classes_selection.h5')
model = load_bert(model_file)

2024-11-08 07:29:47.773571: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-11-08 07:29:47.773603: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
token_dict = get_token_dict()
token_dict


{'': 0,
 '[UNK]': 1,
 '[CLS]': 2,
 '[SEP]': 3,
 '[MASK]': 4,
 'AAA': 5,
 'AAC': 6,
 'AAG': 7,
 'AAT': 8,
 'ACA': 9,
 'ACC': 10,
 'ACG': 11,
 'ACT': 12,
 'AGA': 13,
 'AGC': 14,
 'AGG': 15,
 'AGT': 16,
 'ATA': 17,
 'ATC': 18,
 'ATG': 19,
 'ATT': 20,
 'CAA': 21,
 'CAC': 22,
 'CAG': 23,
 'CAT': 24,
 'CCA': 25,
 'CCC': 26,
 'CCG': 27,
 'CCT': 28,
 'CGA': 29,
 'CGC': 30,
 'CGG': 31,
 'CGT': 32,
 'CTA': 33,
 'CTC': 34,
 'CTG': 35,
 'CTT': 36,
 'GAA': 37,
 'GAC': 38,
 'GAG': 39,
 'GAT': 40,
 'GCA': 41,
 'GCC': 42,
 'GCG': 43,
 'GCT': 44,
 'GGA': 45,
 'GGC': 46,
 'GGG': 47,
 'GGT': 48,
 'GTA': 49,
 'GTC': 50,
 'GTG': 51,
 'GTT': 52,
 'TAA': 53,
 'TAC': 54,
 'TAG': 55,
 'TAT': 56,
 'TCA': 57,
 'TCC': 58,
 'TCG': 59,
 'TCT': 60,
 'TGA': 61,
 'TGC': 62,
 'TGG': 63,
 'TGT': 64,
 'TTA': 65,
 'TTC': 66,
 'TTG': 67,
 'TTT': 68}

In [4]:
file = "/home/anand/projects/workshop/d20.fa"
records = parse_fasta(file)
records

[Record(id='16ea9194-9745-4089-ab62-334c4983b016 runid=e6b015e1f5ce78db0716ea8fba96e399ac9ac8c4 sampleid=8_samples read=1815 ch=78 start_time=2022-07-18T14:04:04Z model_version_id=2021-05-17_dna_r9.4.1_minion_384_d37a2ab9 barcode=barcode07', seq='CGATGTCAATTTGGATGGTCTCTTTGGTCTGTTTAGATGAATTTTCTGAATAAATGTTGAGACCAATTCTTATTATTCTTGGCTTTGATATAGACTACAATATAGTGGCATTGGATCAAACAAGTCTGACAATTTTGATAGCATTAATACTGATAGAATGCAGTTTTTGCCATAGAAGAATGAGAGTTATTGCTATTAATATCTTTGAAAAAAATGTGCAGTTGCAATATTGTGTCTCTGTTGTAATGTGCTTTGAATGAATCCTTTTCATACAGCAGCAGTTCCTACCTACAGAAATTTCAGTTGGTGAGGATACTGACAAAATCTTTTACCAGAAAATACTGTTTCTGCACAGACCCAAGTGACATTGATAGTAGTCATTATATAATGCAGATATAAGGTATATGAAATTCTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGATATACTTCAGTTCAGTTACATTGACACTGATTGATGGGTCTGTGCTACATAGCTGATTTCTGAAAGTCATGTCCGTGTTTAGTCATTCTCCCTTTAGTACAGTTAATGCTGCTCTCTGTTGTGATAGACAATAAACAAACCGCAGCAAGATAAACCTTCCGATAATGGCAGCGTCGCGTGCCACCACTTCCTCCTGCTTTCG

In [5]:
record = records[0]
max_seq_len = 1500
model_max_len = model.input_shape[0][1]
print(model_max_len)
inputs = [seq2tokens(record.seq, token_dict, np.ceil(max_seq_len / 3).astype(int), max_length=model_max_len)]
inputs

print(f'converted sequence {record.id} (length {len(record.seq)}) into {len(inputs)} chunks')


502
converted sequence 16ea9194-9745-4089-ab62-334c4983b016 runid=e6b015e1f5ce78db0716ea8fba96e399ac9ac8c4 sampleid=8_samples read=1815 ch=78 start_time=2022-07-18T14:04:04Z model_version_id=2021-05-17_dna_r9.4.1_minion_384_d37a2ab9 barcode=barcode07 (length 801) into 1 chunks


In [6]:
x = process_bert_tokens_batch(inputs)
print(f'predicting sequence chunks for {record.id}')


predicting sequence chunks for 16ea9194-9745-4089-ab62-334c4983b016 runid=e6b015e1f5ce78db0716ea8fba96e399ac9ac8c4 sampleid=8_samples read=1815 ch=78 start_time=2022-07-18T14:04:04Z model_version_id=2021-05-17_dna_r9.4.1_minion_384_d37a2ab9 barcode=barcode07


In [7]:
batch_size = 32
preds = model.predict(x, verbose=INFO, batch_size=batch_size)
preds

out = []
output_ranks = ['superkingdom', 'phylum', 'genus']
preds = list(map(lambda p: p.mean(axis=0), preds))
annotated = annotate_predictions(preds)
out.append((record.id, [(r, annotated[r]) for r in output_ranks]))
print(f'predicted classes for {len(records)} sequence records')

out

predicted classes for 10 sequence records


[('16ea9194-9745-4089-ab62-334c4983b016 runid=e6b015e1f5ce78db0716ea8fba96e399ac9ac8c4 sampleid=8_samples read=1815 ch=78 start_time=2022-07-18T14:04:04Z model_version_id=2021-05-17_dna_r9.4.1_minion_384_d37a2ab9 barcode=barcode07',
  [('superkingdom',
    {'Archaea': 0.00030615777359344065,
     'Bacteria': 0.001032353611662984,
     'Eukaryota': 0.9981683492660522,
     'Viruses': 0.0004930527647957206,
     'unknown': 1.478750455419231e-10}),
   ('phylum',
    {'Actinobacteria': 1.156473172159167e-05,
     'Apicomplexa': 7.07369763404131e-05,
     'Aquificae': 1.0734114539445727e-06,
     'Arthropoda': 0.0021322881802916527,
     'Artverviricota': 4.6002398448763415e-05,
     'Ascomycota': 0.00015755227650515735,
     'Bacillariophyta': 8.061239350354299e-05,
     'Bacteroidetes': 1.482205971115036e-05,
     'Basidiomycota': 1.298901770496741e-05,
     'Candidatus Thermoplasmatota': 1.0602661859593354e-05,
     'Chlamydiae': 3.369865953573026e-05,
     'Chlorobi': 4.972790520696435e

In [8]:
FIELD_LEN_THR = 50 # prevent overlong IDs

out_values = [['id'] + output_ranks] + [
list(map(lambda x: (x[:FIELD_LEN_THR] + '...' if len(x) > FIELD_LEN_THR
               else x),
         [o[0],     # id
          *[f'{p[0]} ({p[1]:.0%})'
            for p in best_predictions(o[1])] # best prediction per rank
          ])) for o in out]
max_field_lens = [max(o[i] for o in [list(map(len, o)) for o in out_values])
              for i in range(len(out_values[0]))]
row_format = ''.join(f'{{:<{f + 2}}}' for f in max_field_lens)
print('\n'.join(row_format.format(*l) for l in out_values))


id                                                     superkingdom      phylum          genus              
16ea9194-9745-4089-ab62-334c4983b016 runid=e6b015e...  Eukaryota (100%)  Mollusca (94%)  Crassostrea (38%)  
